In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json

# Import API key
import api_keys

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

621

In [3]:
cities=pd.Series(cities)
long=pd.Series(lngs)
lat=pd.Series(lats)

city_data=pd.DataFrame({'city':cities,'Latitude':lat,'Longitude':long})
city_data["Temperature"] = ""
city_data["Humidity"] = ""
city_data["Cloudiness"] = ""
city_data["WindSpeed"] = ""

city_data.head()

,city,Latitude,Longitude,Temperature,Humidity,Cloudiness,WindSpeed
0,san jose,13.319847,-90.940414,,,,
1,uk,54.170687,97.913070,,,,
2,castro,-55.898456,-104.372778,,,,
3,ugoofaaru,6.209589,62.878635,,,,
4,mayumba,-5.758235,8.030540,,,,


## Perform API Calls

In [4]:
# OpenWeatherMap API Key
api_key = api_keys.api_key

# Starting URL for Weather Map API Call
url = f"http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID={api_key}&params"


In [ ]:
for index, row in city_data.iterrows():
    city=row['city']
    lat= row["Latitude"]
    long=row['Longitude']
  
    params= f"lat={lat}&lon={long}"

    response = requests.get(url, params=params).json()
    
    try:
        city_data.loc[index, "Temperature"] = response['main']['temp']
        city_data.loc[index, "Humidity"] =response['main']['humidity']
        city_data.loc[index, "Cloudiness"] = response['clouds']['all']
        city_data.loc[index, "WindSpeed"] =response['wind']['speed']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
city_data.shape

In [ ]:
plt.suptitle(f'Temperature(F) vs. Latitude {time.ctime()}')
plt.scatter(city_data['Latitude'],city_data['Temperature'],alpha=0.95,c='r',edgecolor = 'black', label = 'Temperature')
plt.xlabel('Latitude')
plt.ylabel('Temperature')
plt.ylim(-100,120)
plt.xlim(-100,100)
plt.legend()
plt.grid()
plt.show()

plt.savefig('TempVsLatitude.png')